In [1]:
import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT
from main_utils import *

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
resume_dir = None
resume_iter = False # if True, resume from saved iter_num, otherwise resume from iter_num 0
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = True # if True, always save a checkpoint after each eval
init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_entity = 'ssdd'
wandb_log = True # disabled by default
wandb_project = 'owt'
wandb_run_name = 'gpt2' # 'run' + str(time.time())
exp_name = 'default_exp_name'
# data
train_data_path = 'train_3digit_10000.txt'
gradient_accumulation_steps = 1 # used to simulate larger batch sizes
test_batch_size = 128
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 1024
val_data_path = 'val.bin'
multi_digit = False
num_digit = 5
binary = False
# using two data - data1 = text / data2 = addition
train_both = False # use seperate text/add data for train/val (get_batch uses this to sample from two differernt datasets)
data_ratio = 0.2 # ratio of data_path2 compared with data_path1
train_data_path2 = 'train_addition.bin' # only used when train_both = True
val_data_path2 = 'val_addition.bin'
# evaluation
eval_text = False # if True get perplexity using eval_text_data_path
eval_text_data_path = None # directory to text data (.bin file) - ex. 'data/shakespeare_add_ar_mixed/val_text.bin' 
eval_addition = False # if True compute test accuracy of "a+b="
start = "FILE:data/bal/test_10000.txt"
eval_addition_ar = False
start_ar = None
eval_other = False # use this to evaluate other operations (ex. train on operator '-' but evaluate on other_operator '+')
start_other = None
other_operator = '+'
eval_addition_train = False
start_train = None
reverse_ab = False
reverse_c = False
zero_pad = False
algo_reason = False
add_space = False
# model
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2
bias = False # do we use bias inside LayerNorm and Linear layers?
ckpt_path_name = 'ckpt.pt'
save_final = True
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = None # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster
use_flash = True
data_type='text'
dataset = 'bal'
operator = '+' # can be '+', '-', '*', 'sin', 'sqrt'
data_shuffle = True
data_format = 'plain' # 'plain' or 'reverse' or 'algo_reasoning'
vocabulary = 'all_ascii_chars' # can be 'all_ascii_chars' or 'numbers_only' or 'custom_input_data'
meta_path_specified = False # use saved meta_file (False if data_type='text')
eps = 0
tokenizer = 'char' # by default, use char level tokenizer. but for pretrained models, use openai tokenizer eg: 'gpt2'

simple=False
random_A=False
random_C=False

use_lora = False # use lora (from minLoRA)
print_interval = 2  # if we're using gpt-2 model, I want to see it prompted on text
# -----------------------------------------------------------------------------
# import ipdb; ipdb.set_trace()
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str, type(None)))]
# exec(open('configurator.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------


In [2]:

if min_lr == None:
    min_lr = learning_rate/10

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
print('ddp: ', ddp)
if ddp:
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
else:
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
torch.backends.cudnn.benchmark = True # cudnn auto-tuner
torch.backends.cudnn.deterministic = False # cudnn auto-tuner

device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
if data_type == 'binary':
    data_dir = os.path.join('data', dataset)
    train_data = np.memmap(os.path.join(data_dir, train_data_path), dtype=np.uint16, mode='r')
    val_data = np.memmap(os.path.join(data_dir, val_data_path), dtype=np.uint16, mode='r')
    if train_both:
        train_data2 = np.memmap(os.path.join(data_dir, train_data_path2), dtype=np.uint16, mode='r')
        val_data2 = np.memmap(os.path.join(data_dir, val_data_path2), dtype=np.uint16, mode='r')
    if eval_text:
        if eval_text_data_path is None:
            print('eval_text_data_path is None!!! No binary file to evaluate perplexity on.') 
        eval_text_data = np.memmap(eval_text_data_path, dtype=np.uint16, mode='r')
    # test_data_str = None # test_data for addition testing will be handled with "start"
    meta_path = None
else:
    # check for data_format
    if data_type == 'text':
        if (data_format == 'reverse' and not reverse_c) or (reverse_c and data_format != 'reverse'):
            raise ValueError('reverse_c must be True for data_format == "reverse"')
        elif (data_format == 'algo_reasoning' and not algo_reason) or (algo_reason and data_format != 'algo_reasoning'):
            raise ValueError('algo_reason must be True for data_format == "algo_reasoning"')
    meta_path_specified = False

    data_dir = os.path.join('data', dataset)
    train_data_path = os.path.join(data_dir, train_data_path)
    # val_data = os.path.join(data_dir, val_data_path)
    train_data_list = get_data_list(train_data_path, operator=operator)
    val_data_list = get_data_list(filename=None, operator=operator) # get_data_list(val_data, operator='+')
    train_data_str = generate_data_str(train_data_list, operator=operator, format=data_format, train=True, shuffle=data_shuffle, add_space=add_space, simple=simple, random_A=random_A, random_C=random_C)
    val_data_str = generate_data_str(val_data_list, operator=operator, format=data_format, train=True, shuffle=data_shuffle, add_space=add_space, simple=simple, random_A=random_A, random_C=random_C)
    meta, meta_path, data_encoder, data_decoder = create_meta_file(vocabulary=vocabulary, input_data_str=train_data_str, tokenizer=tokenizer)
    meta_vocab_size = meta['vocab_size']
    train_data = data_encoder(train_data_str)
    val_data = data_encoder(val_data_str)
    if eval_addition_train and start_train is None:
        # specify the start_train to be oour train data file
        start_train = f"FILE:{train_data_path}"
        

def get_batch(split):
    data = train_data if split == 'train' else val_data
    if train_both:
        data2 = train_data2 if split == 'train' else val_data2
        batch_size2 = int(batch_size*data_ratio)
        ix = torch.randint(len(data) - block_size, (batch_size-batch_size2,))
        ix2 = torch.randint(len(data2) - block_size, (batch_size2,))
    else:
        ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if train_both:
        x2 = torch.stack([torch.from_numpy((data2[i:i+block_size]).astype(np.int64)) for i in ix2])
        y2 = torch.stack([torch.from_numpy((data2[i+1:i+1+block_size]).astype(np.int64)) for i in ix2])
        x = torch.cat([x,x2])
        y = torch.cat([y,y2])    

    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9
best_perplexity = 1e9 # on text data
best_accuracy = -1 # on addition data

if meta_path_specified:
    # attempt to derive vocab_size from the dataset
    meta_path = os.path.join(data_dir, 'meta.pkl')
    meta_vocab_size = None
    if os.path.exists(meta_path):
        with open(meta_path, 'rb') as f:
            meta = pickle.load(f)
        meta_vocab_size = meta['vocab_size']
        print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")
    else:
        meta_path = None

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line


# init a new model from scratch
print("Initializing a new model from scratch")
# determine the vocab size we'll use for from-scratch training
if meta_vocab_size is None:
    print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)

encode, decode = get_encode_decode(meta_path, tokenizer=tokenizer)


ddp:  False
Creating meta file for all reasonable characters...
all the unique characters: 
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~
vocab size: 96
data has 119,890 tokens
data has 12,244 tokens
Initializing a new model from scratch
Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
number of parameters: 10.66M
Loading meta from meta_all_ascii_chars.pkl...


In [3]:
def load_checkpoint(ckpt_path, model):
        # load ckpt into model
        checkpoint = torch.load(ckpt_path, map_location=device)

        checkpoint_model_args = checkpoint['model_args']
        for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
                model_args[k] = checkpoint_model_args[k]
        # create the model
        gptconf = GPTConfig(**model_args)
        model = GPT(gptconf)
        state_dict = checkpoint['model']
        # fix the keys of the state dictionary :(
        # honestly no idea how checkpoints sometimes get this prefix, have to debug more
        unwanted_prefix = '_orig_mod.'
        for k,v in list(state_dict.items()):
                if k.startswith(unwanted_prefix):
                        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
        model.load_state_dict(state_dict)
        return model

In [4]:
def generate_output(model, prompt, max_new_tokens=5):
    temperature = 0.8
    top_k = 200

    model.eval()
    model.to(device)
    if compile:
        model = torch.compile(model) # requires PyTorch 2.0 (optional)
    # run generation
    
    start_ids = encode(prompt)
    x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

    with torch.no_grad():
        with ctx:
            num_samples = 1
            for k in range(num_samples):
                y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
                # print(decode(y[0].tolist()))
                # print('---------------')
    return decode(y[0].tolist())


# To-DO: 
1 or 2 wrong character in output (only select test data with output size 2+ or 3+) -> measure accuracy drop

In [5]:
def get_test_data(data_type=data_type, start=start):
    if data_type=='text':
        test_data_file = start[5:]
        print(f"Evaluating Addition using test data file: {test_data_file}")
        # we know test examples are test.txt
        test_data_list = get_data_list(test_data_file, operator=operator)
        test_data_str = generate_data_str(test_data_list, operator=operator, format=data_format, train=False, shuffle=True, add_space=add_space, simple=simple, random_A=random_A, random_C=random_C)
        if algo_reason:
            lines = [x.strip() + "\nTarget:\n" for x in test_data_str.split("Target:")]
            lines = lines[:-1]
        else:
            lines = test_data_str.split('\n')[:-1]
    else:
        # encode the beginning of the prompt
        if start.startswith('FILE:'):
            with open(start[5:], 'r', encoding='utf-8') as f:
                # start = f.read()
                lines = [line.rstrip() for line in f]
                if algo_reason:
                    lines2 = [lines[2*i]+'\n'+lines[2*i+1]+'\n' for i in range(len(lines)//2)]
                    lines = lines2
        else:
            lines = start.splitlines()
            if algo_reason:
                lines2 = [lines[2*i]+'\n'+lines[2*i+1]+'\n' for i in range(len(lines)//2)]
                lines = lines2
    
    return lines

In [6]:
lines = get_test_data(data_type=data_type, start=start)
print(lines[:10])
a,b,c,op = get_abc_new(lines[0], zero_pad=zero_pad, reverse_ab=reverse_ab, binary=binary, few_shot=False, algo_reason=algo_reason)
print(a,b,c,op)

Evaluating Addition using test data file: data/bal/test_10000.txt
['816+351=', '99+221=', '730+820=', '124+127=', '39+257=', '135+532=', '674+773=', '345+689=', '863+16=', '949+318=']
816 351 1167 +


In [7]:
def randomize_c(c_str, precise=True, reverse_c=False):
    num_digit = len(c_str)
    c = int(c_str)
    if precise:
        if not reverse_c:
            if random.random() < 0.5:
                return str(c+1)
            else:
                return str(c-1)
        else:
            last_c = c_str[-1]
            if random.random() < 0.5:
                return c_str[:-1] + str( (int(last_c)+1) % 10)
            else:
                return c_str[:-1] + str( (int(last_c)-1) % 10)

    else:
        # return a random number with same number of digits that is not c
        while True:
            c = np.random.randint(10**(num_digit-1), 10**num_digit -1)
            if c != int(c_str):
                return str(c)

a = '9'
print(a, randomize_c(a, reverse_c=True))
print(a, randomize_c(a, reverse_c=True))
print(a, randomize_c(a, reverse_c=False))
print(a, randomize_c(a, reverse_c=False))

9 0
9 8
9 8
9 10


In [8]:
def test_model(model, reverse_c = False, precise=True, noise_digit=1):
    total = 0
    correct = 0
    within_1 = 0

    for line in lines:
        a,b,c,op = get_abc_new(line, zero_pad=zero_pad, reverse_ab=reverse_ab, binary=binary, few_shot=False, algo_reason=algo_reason)
        c_str = str(c)
        # if len(c_str) <= noise_digit:
        if len(str(a)) < 3 or len(str(b)) < 3:
            continue
        
        total += 1
        if reverse_c:
            c_str = c_str[::-1]
        
        next_c = c_str[noise_digit] # next output to be expected
        noise_line = line + randomize_c(c_str[:noise_digit], precise = precise, reverse_c = reverse_c)
        output = generate_output(model, noise_line, max_new_tokens=1)
        if output[-1] == next_c:
            correct += 1
        else:
            if output[-1] == str( (int(next_c)+1) % 10) or output[-1] == str( (int(next_c)-1) % 10):
                within_1 += 1
                print(f"within 1!")
            print(f"(With Noise) input: {noise_line} | output: {output}")
            output = generate_output(model, line + c_str[:noise_digit], max_new_tokens=1)
            print(f"(No Noise) input: {line+c_str[:noise_digit]} | output: {output}")
            print(f"True: {line+str(c)}")
        
    print(f"Accuracy: {correct}/{total} ({correct/total*100}%)")
    print(f"Within 1: {correct+within_1}/{total} ({(correct + within_1)/total*100}%)")


In [9]:
data_type='text'
data_format='plain'
operator='+'
dataset = 'bal'
batch_size = 256
block_size = 256 # context of up to 256 previous characters
train_data_path = 'train_3digit_10000.txt'
# val_data_path = 'val.bin'
ckpt_path_name = 'out2/addition_plain/ckpt_final.pt'
eval_addition = True
reverse_c = False
start = "FILE:data/bal/test_10000.txt"
eval_addition_train = True


In [10]:
ckpt = "out2/addition_plain/ckpt_final.pt"

model = load_checkpoint(ckpt, model)

start = 'FILE:data/bal/test_10000.txt'
lines = get_test_data(data_type=data_type, start=start)
print(lines[:10])

prompt = "et tu brute"
print(generate_output(model, prompt, max_new_tokens=1))

prompt = "198+843="
print(generate_output(model, prompt, max_new_tokens=4))


Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
number of parameters: 10.66M
Evaluating Addition using test data file: data/bal/test_10000.txt
['98+805=', '440+766=', '614+850=', '457+799=', '154+57=', '682+479=', '465+814=', '119+376=', '419+206=', '490+189=']
et tu brute+
198+843=1041


In [13]:
test_model(model, reverse_c=reverse_c, precise=True, noise_digit=2)

within 1!
(With Noise) input: 758+251=9 | output: 758+251=91
(No Noise) input: 758+251=10 | output: 758+251=100
True: 758+251=1009
within 1!
(With Noise) input: 682+407=9 | output: 682+407=99
(No Noise) input: 682+407=10 | output: 682+407=108
True: 682+407=1089
within 1!
(With Noise) input: 335+694=9 | output: 335+694=93
(No Noise) input: 335+694=10 | output: 335+694=102
True: 335+694=1029
within 1!
(With Noise) input: 919+417=12 | output: 919+417=124
(No Noise) input: 919+417=13 | output: 919+417=134
True: 919+417=1336
within 1!
(With Noise) input: 349+711=9 | output: 349+711=95
(No Noise) input: 349+711=10 | output: 349+711=106
True: 349+711=1060
within 1!
(With Noise) input: 958+122=9 | output: 958+122=97
(No Noise) input: 958+122=10 | output: 958+122=108
True: 958+122=1080
within 1!
(With Noise) input: 821+239=9 | output: 821+239=95
(No Noise) input: 821+239=10 | output: 821+239=106
True: 821+239=1060
within 1!
(With Noise) input: 100+939=9 | output: 100+939=94
(No Noise) input: 10

In [14]:
test_model(model, reverse_c=reverse_c, precise=False, noise_digit=2)

(With Noise) input: 614+850=64 | output: 614+850=644
(No Noise) input: 614+850=14 | output: 614+850=146
True: 614+850=1464
within 1!
(With Noise) input: 457+799=38 | output: 457+799=386
(No Noise) input: 457+799=12 | output: 457+799=125
True: 457+799=1256
(With Noise) input: 682+479=96 | output: 682+479=961
(No Noise) input: 682+479=11 | output: 682+479=116
True: 682+479=1161
(With Noise) input: 465+814=63 | output: 465+814=639
(No Noise) input: 465+814=12 | output: 465+814=127
True: 465+814=1279
(With Noise) input: 119+376=16 | output: 119+376=169
(No Noise) input: 119+376=49 | output: 119+376=495
True: 119+376=495
(With Noise) input: 159+445=19 | output: 159+445=190
(No Noise) input: 159+445=60 | output: 159+445=604
True: 159+445=604
(With Noise) input: 791+768=63 | output: 791+768=639
(No Noise) input: 791+768=15 | output: 791+768=155
True: 791+768=1559
(With Noise) input: 976+904=39 | output: 976+904=390
(No Noise) input: 976+904=18 | output: 976+904=188
True: 976+904=1880
(With No

# Reverse

In [15]:
data_type='text'
data_format='reverse'
operator='+'
dataset = 'bal'
batch_size = 256
block_size = 256 # context of up to 256 previous characters
train_data_path = 'train_3digit_10000.txt'
# val_data_path = 'val.bin'
ckpt_path_name = 'out2/addition_reverse/ckpt_10000_final.pt'
reverse_c = True
eval_addition = True
start = "FILE:data/bal/test_10000.txt"
eval_addition_train = True

In [16]:
model = load_checkpoint(ckpt_path_name, model)

lines = get_test_data(data_type=data_type, start=start)
print(lines[:10])

Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
Using Flash Attention
number of parameters: 10.66M
Evaluating Addition using test data file: data/bal/test_10000.txt
['$198+899=', '$987+255=', '$369+399=', '$468+763=', '$530+409=', '$649+466=', '$448+782=', '$740+328=', '$334+476=', '$460+982=']


In [17]:
test_model(model, reverse_c=reverse_c, precise=True, noise_digit=2)

within 1!
(With Noise) input: $880+816=60 | output: $880+816=607
(No Noise) input: $880+816=69 | output: $880+816=696
True: $880+816=1696
within 1!
(With Noise) input: $346+351=70 | output: $346+351=707
(No Noise) input: $346+351=79 | output: $346+351=796
True: $346+351=697
within 1!
(With Noise) input: $766+133=90 | output: $766+133=909
(No Noise) input: $766+133=99 | output: $766+133=998
True: $766+133=899
within 1!
(With Noise) input: $467+138=59 | output: $467+138=595
(No Noise) input: $467+138=50 | output: $467+138=506
True: $467+138=605
within 1!
(With Noise) input: $520+473=30 | output: $520+473=300
(No Noise) input: $520+473=39 | output: $520+473=399
True: $520+473=993
within 1!
(With Noise) input: $862+840=29 | output: $862+840=296
(No Noise) input: $862+840=20 | output: $862+840=207
True: $862+840=1702
within 1!
(With Noise) input: $217+481=80 | output: $217+481=807
(No Noise) input: $217+481=89 | output: $217+481=896
True: $217+481=698
within 1!
(With Noise) input: $382+219=

In [18]:
test_model(model, reverse_c=reverse_c, precise=False, noise_digit=2)

within 1!
(With Noise) input: $147+728=50 | output: $147+728=509
(No Noise) input: $147+728=57 | output: $147+728=578
True: $147+728=875
within 1!
(With Noise) input: $261+626=72 | output: $261+626=729
(No Noise) input: $261+626=78 | output: $261+626=788
True: $261+626=887
within 1!
(With Noise) input: $826+996=19 | output: $826+996=197
(No Noise) input: $826+996=22 | output: $826+996=228
True: $826+996=1822
within 1!
(With Noise) input: $171+120=23 | output: $171+120=233
(No Noise) input: $171+120=19 | output: $171+120=192
True: $171+120=291
within 1!
(With Noise) input: $520+473=33 | output: $520+473=330
(No Noise) input: $520+473=39 | output: $520+473=399
True: $520+473=993
within 1!
(With Noise) input: $862+840=67 | output: $862+840=676
(No Noise) input: $862+840=20 | output: $862+840=207
True: $862+840=1702
within 1!
(With Noise) input: $173+574=89 | output: $173+574=896
(No Noise) input: $173+574=74 | output: $173+574=747
True: $173+574=747
within 1!
(With Noise) input: $584+744=

within 1!
(With Noise) input: $480+909=51 | output: $480+909=514
(No Noise) input: $480+909=98 | output: $480+909=983
True: $480+909=1389
within 1!
(With Noise) input: $319+831=10 | output: $319+831=102
(No Noise) input: $319+831=05 | output: $319+831=051
True: $319+831=1150
within 1!
(With Noise) input: $673+153=38 | output: $673+153=387
(No Noise) input: $673+153=62 | output: $673+153=628
True: $673+153=826
within 1!
(With Noise) input: $686+428=56 | output: $686+428=560
(No Noise) input: $686+428=41 | output: $686+428=411
True: $686+428=1114
within 1!
(With Noise) input: $310+793=27 | output: $310+793=270
(No Noise) input: $310+793=30 | output: $310+793=301
True: $310+793=1103
within 1!
(With Noise) input: $968+603=40 | output: $968+603=406
(No Noise) input: $968+603=17 | output: $968+603=175
True: $968+603=1571
within 1!
(With Noise) input: $760+828=31 | output: $760+828=316
(No Noise) input: $760+828=88 | output: $760+828=885
True: $760+828=1588
within 1!
(With Noise) input: $957+